In [1]:
from settings import *

from src.data.preprocessing_v0 import preprocess, features_input


import torch
from torch import nn

import pandas as pd
import joblib

from sklearn.preprocessing import MinMaxScaler

from typing import Dict, Union, Callable


In [12]:
df = pd.read_csv("submissions/58ce7d6f-2a12-412a-a989-a446f1990dec.csv")

df['longitude_predicted'] = df['longitude_predicted'].apply(lambda x: (x + 180) % 360 - 180)
df['latitude_predicted'] = df['latitude_predicted'].apply(lambda x: (x + 90) % 180 - 90)

In [13]:
df.to_csv("submissions/258ce7d6f-2a12-412a-a989-a446f1990dec.csv", index=False)

In [2]:
from src.model.transformer_enc import EncoderModel

In [3]:
model = EncoderModel()

model_scripted = torch.jit.script(model)
model_scripted.save(MODEL_FOLDER.joinpath("test.pt")) 

/Users/arthurtestard/envs/ntnu/aitask3/lib/python3.10/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.activation_relu_or_gelu was not True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [2]:

ais_train = pd.read_csv(AIS_TRAIN, sep='|')
ais_train['time'] = pd.to_datetime(ais_train['time'])

ais_test = pd.read_csv(AIS_TEST, sep=",")
ais_test['time'] = pd.to_datetime(ais_test['time']) 


In [9]:
ais_train['navstat'].isna().sum()

0

In [10]:
from src.data.features_v0 import one_hot_encode
    # COPY IN CASE
df_train = ais_train.copy()

# test_vessel_ids = list(df_test["vesselId"].unique())

# CONCAT train AND test SETS
# if verbose:
#     print("Concat train and test sets...")
# df = concat_train_test_sets(df_train, df_test, features_raw)

# CLEAN DATA (future)
# - look for options (z-score?)
# - data augmentation (add samples when time_diff > threshold)

# CREATE FEATURES (can be automated in the future by passing the functions as args)
# if verbose:
#     print("Create features...")
# df = create_time_diff_feature(df)
# df = create_long_lat_diff_feature(df)

df = one_hot_encode(df_train, "navstat")

ImportError: cannot import name 'one_hot_encode' from 'src.data.features_v0' (/Users/arthurtestard/vessel_position_forecast/src/data/features_v0.py)

In [5]:
df.columns

Index(['time', 'cog', 'sog', 'rot', 'heading', 'navstat', 'etaRaw', 'latitude',
       'longitude', 'vesselId', 'portId', 'navstat_0', 'navstat_1',
       'navstat_2', 'navstat_3', 'navstat_4', 'navstat_5', 'navstat_6',
       'navstat_7', 'navstat_8', 'navstat_9', 'navstat_11', 'navstat_12',
       'navstat_13', 'navstat_14', 'navstat_15'],
      dtype='object')

In [21]:
import torch
from src.model.ffn import FFNModel

model = FFNModel()

for layer in model.main:
    if isinstance(layer, nn.Linear):
        torch.nn.init.xavier_normal(layer.weight)

/var/folders/dv/nzlg6jm168311nq7f93wfrd80000gn/T/ipykernel_81752/2810861855.py:8: FutureWarning: `nn.init.xavier_normal` is now deprecated in favor of `nn.init.xavier_normal_`.
  torch.nn.init.xavier_normal(layer.weight)


In [17]:
model.model

Linear(in_features=7, out_features=128, bias=True)

In [8]:
list(range(10)) * 2

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [6]:
seq_len = 48
do_preprocess = False

In [4]:
def pipeline(seq_len = 48, do_preprocess = True):
    if do_preprocess:
        ais_train = pd.read_csv(AIS_TRAIN, sep='|')
        ais_train['time'] = pd.to_datetime(ais_train['time'])

        ais_test = pd.read_csv(AIS_TEST, sep=",")
        ais_test['time'] = pd.to_datetime(ais_test['time']) 

        X_train, X_val, y_train, y_val, test_set, scaler, dropped_vessel_ids = preprocess(
            ais_train, 
            ais_test,
            seq_type="n_in_1_out",
            seq_len=seq_len,
            seq_len_out=1,
            verbose=True,
            to_torch=True,
            parallelize_seq = True,
            scaler=MinMaxScaler()
        )

        X_train = torch.Tensor(X_train)
        y_train = torch.Tensor(y_train)

        X_val = torch.Tensor(X_val)
        y_val = torch.Tensor(y_val)

        torch.save(X_train, LAST_PREPROCESS_FOLDER.joinpath("X_train.pt"))
        torch.save(y_train, LAST_PREPROCESS_FOLDER.joinpath("y_train.pt"))
        torch.save(X_val, LAST_PREPROCESS_FOLDER.joinpath("X_val.pt"))
        torch.save(y_val, LAST_PREPROCESS_FOLDER.joinpath("y_val.pt"))

        joblib.dump(scaler, LAST_PREPROCESS_FOLDER.joinpath("scaler")) 
        test_set.to_csv(LAST_PREPROCESS_FOLDER.joinpath("test_set.csv"))

    else:
        try:
            
            X_train = torch.load(LAST_PREPROCESS_FOLDER.joinpath("X_train.pt"), weights_only=True)
            y_train = torch.load(LAST_PREPROCESS_FOLDER.joinpath("y_train.pt"), weights_only=True)
            X_val = torch.load(LAST_PREPROCESS_FOLDER.joinpath("X_val.pt"), weights_only=True)
            y_val = torch.load(LAST_PREPROCESS_FOLDER.joinpath("y_val.pt"), weights_only=True)

            scaler = joblib.load(LAST_PREPROCESS_FOLDER.joinpath("scaler")) 
            test_set = pd.read_csv(LAST_PREPROCESS_FOLDER.joinpath("test_set.csv"))
            
            dropped_vessel_ids = ['61e9f3adb937134a3c4bfe37', '61e9f3cbb937134a3c4bff09']

        except:
            print(f"ERROR: File missing in {str(LAST_PREPROCESS_FOLDER)}. Now run preprocessing...")
            return pipeline(seq_len=seq_len, do_preprocess=True, paralize_preprocess=True)
        
    return X_train, X_val, y_train, y_val, test_set, scaler, dropped_vessel_ids


In [24]:
ais_train["navstat"].value_counts()

navstat
0     782099
5     605831
1     129515
2       2081
8        948
15       670
6        521
4        232
3        161
11         2
12         1
13         1
14         1
7          1
9          1
Name: count, dtype: int64

In [45]:

schedules = pd.read_csv(SCHEDULES_TO_MAY_2024, sep="|")

In [32]:
type(schedules["vesselId"].iloc[0])

str

In [48]:
ports = pd.read_csv(PORTS, sep="|")

In [49]:
ports.isna().sum()

portId          0
name            0
portLocation    0
longitude       0
latitude        0
UN_LOCODE       0
countryName     0
ISO             0
dtype: int64

In [36]:
ais_train.head()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,01-09 23:00,-34.74370,-57.85130,61e9f3a8b937134a3c4bfdf7,61d371c43aeaecc07011a37f
1,2024-01-01 00:00:36,109.6,0.0,-6,347,1,12-29 20:00,8.89440,-79.47939,61e9f3d4b937134a3c4bff1f,634c4de270937fc01c3a7689
2,2024-01-01 00:01:45,111.0,11.0,0,112,0,01-02 09:00,39.19065,-76.47567,61e9f436b937134a3c4c0131,61d3847bb7b7526e1adf3d19
3,2024-01-01 00:03:11,96.4,0.0,0,142,1,12-31 20:00,-34.41189,151.02067,61e9f3b4b937134a3c4bfe77,61d36f770a1807568ff9a126
4,2024-01-01 00:03:51,214.0,19.7,0,215,0,01-25 12:00,35.88379,-5.91636,61e9f41bb937134a3c4c0087,634c4de270937fc01c3a74f3


In [37]:
schedules.head()

,vesselId,shippingLineId,shippingLineName,arrivalDate,sailingDate,portName,portId,portLatitude,portLongitude
0,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-02 00:00:00+00:00,2023-10-03 00:00:00+00:00,Port of Brunswick,61d38499b7b7526e1adf3d54,31.140556,-81.496667
1,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-27 00:00:00+00:00,2023-10-27 00:00:00+00:00,Port of Southampton,61d3832bb7b7526e1adf3b63,50.902500,-1.428889
2,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-19 00:00:00+00:00,2023-10-20 00:00:00+00:00,Port of Bremerhaven,61d375e793c6feb83e5eb3e2,53.563611,8.554722
3,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-09 00:00:00+00:00,2023-10-10 00:00:00+00:00,Port of New York,61d38481b7b7526e1adf3d23,40.688333,-74.028611
4,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-09-25 00:00:00+00:00,2023-09-26 00:00:00+00:00,Manzanillo International Terminal,61d37d0199db2ccf7339eee1,9.372370,-79.879790


In [51]:
ais_train.sort_values(by="time")

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,01-09 23:00,-34.74370,-57.85130,61e9f3a8b937134a3c4bfdf7,61d371c43aeaecc07011a37f
1,2024-01-01 00:00:36,109.6,0.0,-6,347,1,12-29 20:00,8.89440,-79.47939,61e9f3d4b937134a3c4bff1f,634c4de270937fc01c3a7689
2,2024-01-01 00:01:45,111.0,11.0,0,112,0,01-02 09:00,39.19065,-76.47567,61e9f436b937134a3c4c0131,61d3847bb7b7526e1adf3d19
3,2024-01-01 00:03:11,96.4,0.0,0,142,1,12-31 20:00,-34.41189,151.02067,61e9f3b4b937134a3c4bfe77,61d36f770a1807568ff9a126
4,2024-01-01 00:03:51,214.0,19.7,0,215,0,01-25 12:00,35.88379,-5.91636,61e9f41bb937134a3c4c0087,634c4de270937fc01c3a74f3
...,...,...,...,...,...,...,...,...,...,...,...
1522055,2024-05-07 23:59:07,25.1,2.8,2,22,0,05-12 01:00,41.33699,2.15130,61e9f3a2b937134a3c4bfdd7,61d37f9c29b60f6113c89e65
1522062,2024-05-07 23:59:08,269.8,14.9,-1,270,0,05-15 23:00,49.71372,-5.22042,61e9f43db937134a3c4c0169,634c4de270937fc01c3a787b
1522063,2024-05-07 23:59:08,8.0,18.7,0,6,0,05-08 12:45,38.27895,10.78280,61e9f469b937134a3c4c029b,61d3781293c6feb83e5eb73b
1522061,2024-05-07 23:59:08,12.3,17.1,0,13,0,05-10 03:00,38.96142,-12.00502,61e9f3aeb937134a3c4bfe43,634c4de270937fc01c3a76a1


In [47]:
len(schedules["vesselId"].unique())

252

In [82]:
schedules_ft = schedules.copy()[["vesselId","shippingLineId","arrivalDate","sailingDate","portLatitude","portLongitude",]]
schedules_ft.head()


,vesselId,shippingLineId,arrivalDate,sailingDate,portLatitude,portLongitude
0,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,2023-10-02 00:00:00+00:00,2023-10-03 00:00:00+00:00,31.140556,-81.496667
1,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,2023-10-27 00:00:00+00:00,2023-10-27 00:00:00+00:00,50.902500,-1.428889
2,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,2023-10-19 00:00:00+00:00,2023-10-20 00:00:00+00:00,53.563611,8.554722
3,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,2023-10-09 00:00:00+00:00,2023-10-10 00:00:00+00:00,40.688333,-74.028611
4,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,2023-09-25 00:00:00+00:00,2023-09-26 00:00:00+00:00,9.372370,-79.879790


In [83]:
schedules_ft.isna().sum()

vesselId          4615
shippingLineId       0
arrivalDate       5515
sailingDate       1881
portLatitude      4402
portLongitude     4402
dtype: int64

In [23]:
from src.model.ffn import FFNModel

isinstance(FFNModel(), nn.Module)

True

In [4]:
from src.data.features import create_rot_features

df = create_rot_features(ais_train)
df

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId,rot_calculated,rot_category,rot_category_left_more_than_5_deg_per_30s,rot_category_no_turn,rot_category_right_more_than_5_deg_per_30s,rot_category_turning_left,rot_category_turning_right
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,01-09 23:00,-34.74370,-57.85130,61e9f3a8b937134a3c4bfdf7,61d371c43aeaecc07011a37f,0.000000,no_turn,0.0,1.0,0.0,0.0,0.0
1,2024-01-01 00:00:36,109.6,0.0,-6,347,1,12-29 20:00,8.89440,-79.47939,61e9f3d4b937134a3c4bff1f,634c4de270937fc01c3a7689,-11.593435,turning_left,0.0,0.0,0.0,1.0,0.0
2,2024-01-01 00:01:45,111.0,11.0,0,112,0,01-02 09:00,39.19065,-76.47567,61e9f436b937134a3c4c0131,61d3847bb7b7526e1adf3d19,0.000000,no_turn,0.0,1.0,0.0,0.0,0.0
3,2024-01-01 00:03:11,96.4,0.0,0,142,1,12-31 20:00,-34.41189,151.02067,61e9f3b4b937134a3c4bfe77,61d36f770a1807568ff9a126,0.000000,no_turn,0.0,1.0,0.0,0.0,0.0
4,2024-01-01 00:03:51,214.0,19.7,0,215,0,01-25 12:00,35.88379,-5.91636,61e9f41bb937134a3c4c0087,634c4de270937fc01c3a74f3,0.000000,no_turn,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1522060,2024-05-07 23:59:07,359.1,13.4,0,1,0,05-08 05:00,52.19131,-5.82223,clh6aqawa0002gh0zypfa5dut,634c4de270937fc01c3a7417,0.000000,no_turn,0.0,1.0,0.0,0.0,0.0
1522061,2024-05-07 23:59:08,12.3,17.1,0,13,0,05-10 03:00,38.96142,-12.00502,61e9f3aeb937134a3c4bfe43,634c4de270937fc01c3a76a1,0.000000,no_turn,0.0,1.0,0.0,0.0,0.0
1522062,2024-05-07 23:59:08,269.8,14.9,-1,270,0,05-15 23:00,49.71372,-5.22042,61e9f43db937134a3c4c0169,634c4de270937fc01c3a787b,-4.733000,turning_left,0.0,0.0,0.0,1.0,0.0
1522063,2024-05-07 23:59:08,8.0,18.7,0,6,0,05-08 12:45,38.27895,10.78280,61e9f469b937134a3c4c029b,61d3781293c6feb83e5eb73b,0.000000,no_turn,0.0,1.0,0.0,0.0,0.0


In [26]:
df = ais_train.copy()
df['time_diff'] = (
        df
        .sort_values(by=['time'])
        .groupby("vesselId")['time']
        .diff(1)
        .abs()
        .dropna()
        .dt.total_seconds()
        .astype(int)
    )

In [29]:
df.sort_values(by=['vesselId', 'time' ])

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId,rot_calculated,rot_category,time_diff
131115,2024-01-12 14:07:47,308.1,17.1,-6,316,0,01-08 06:00,7.50361,77.58340,61e9f38eb937134a3c4bfd8b,61d376b393c6feb83e5eb50c,-11.593435,turning_left,NaN
131279,2024-01-12 14:31:00,307.6,17.3,5,313,0,01-14 23:30,7.57302,77.49505,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546,10.583310,turning_right,1393.0
131514,2024-01-12 14:57:23,306.8,16.9,5,312,0,01-14 23:30,7.65043,77.39404,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546,10.583310,turning_right,1583.0
131696,2024-01-12 15:18:48,307.9,16.9,6,313,0,01-14 23:30,7.71275,77.31394,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546,11.593435,turning_right,1285.0
131885,2024-01-12 15:39:47,307.0,16.3,7,313,0,01-14 23:30,7.77191,77.23585,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546,12.522341,turning_right,1259.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521244,2024-05-07 22:36:16,324.1,13.5,-2,325,0,05-08 03:00,59.63337,21.43237,clh6aqawa0007gh0z9h6zi9bo,61d373b83aeaecc07011a62b,-6.693473,turning_left,1259.0
1521409,2024-05-07 22:57:05,324.2,13.3,-3,326,0,05-08 03:00,59.69588,21.34225,clh6aqawa0007gh0z9h6zi9bo,61d373b83aeaecc07011a62b,-8.197796,turning_left,1249.0
1521625,2024-05-07 23:17:54,356.5,12.2,-1,354,0,05-08 03:00,59.76388,21.35317,clh6aqawa0007gh0z9h6zi9bo,61d373b83aeaecc07011a62b,-4.733000,turning_left,1249.0
1521821,2024-05-07 23:38:13,52.6,17.3,3,50,0,05-08 03:00,59.83316,21.38489,clh6aqawa0007gh0z9h6zi9bo,61d373b83aeaecc07011a62b,8.197796,turning_right,1219.0


In [32]:
ais_test["vesselId"].nunique()

215

In [13]:
ais_train[ais_train["rot"]==128]

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId,rot_calculated,rot_category
6,2024-01-01 00:05:40,123.4,0.0,128,511,5,12-16 01:00,40.71466,29.46603,61e9f46bb937134a3c4c02b3,61d38259b7b7526e1adf3a41,53.547782,turning_right
221,2024-01-01 00:35:42,123.4,0.0,128,511,5,12-16 01:00,40.71467,29.46602,61e9f46bb937134a3c4c02b3,61d38259b7b7526e1adf3a41,53.547782,turning_right
306,2024-01-01 00:53:39,123.4,0.0,128,511,5,12-16 01:00,40.71467,29.46604,61e9f46bb937134a3c4c02b3,61d38259b7b7526e1adf3a41,53.547782,turning_right
532,2024-01-01 01:17:43,123.4,0.0,128,511,5,12-16 01:00,40.71467,29.46603,61e9f46bb937134a3c4c02b3,61d38259b7b7526e1adf3a41,53.547782,turning_right
614,2024-01-01 01:35:41,123.4,0.0,128,511,5,12-16 01:00,40.71467,29.46602,61e9f46bb937134a3c4c02b3,61d38259b7b7526e1adf3a41,53.547782,turning_right
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515311,2024-05-07 12:15:55,44.6,1.0,128,33,0,03-05 22:30,36.13065,-5.43264,61e9f469b937134a3c4c0297,61d37f9b29b60f6113c89e64,53.547782,turning_right
1515888,2024-05-07 13:18:21,226.6,2.2,128,216,0,01-01 03:00,37.84410,23.52665,61e9f3bcb937134a3c4bfe91,634c4de270937fc01c3a73b2,53.547782,turning_right
1517516,2024-05-07 16:16:57,68.5,7.6,128,55,0,05-01 23:00,14.69897,120.18389,61e9f473b937134a3c4c02df,61d37d5799db2ccf7339ef3b,53.547782,turning_right
1518297,2024-05-07 17:39:09,275.3,0.5,128,218,0,05-07 02:00,-27.37681,153.16453,6326f5a6c46d6a20d22ca31e,61d36f640a1807568ff9a103,53.547782,turning_right


In [36]:
from sklearn.preprocessing import StandardScaler
from src.data.preprocessing import preprocess

do_preprocess = True
seq_len = 8

ais_train = pd.read_csv(AIS_TRAIN, sep='|')
ais_train['time'] = pd.to_datetime(ais_train['time'])

ais_test = pd.read_csv(AIS_TEST, sep=",")
ais_test['time'] = pd.to_datetime(ais_test['time']) 
preprocess_file = Path(f"data/preprocessed_v2_seq_{seq_len}/")

if do_preprocess:
    X_train, X_val, y_train, y_val, test_set, scaler, df_lat_long, dropped_vessel_ids = preprocess(
        ais_train, 
        ais_test,
        seq_type="n_in_1_out",
        seq_len=seq_len,
        seq_len_out=1,
        verbose=True,
        to_torch=True,
        parallelize_seq=True,
        scaler=StandardScaler(),
        preprocess_folder=preprocess_file
    )

Concat train and test sets...
1573804
df.isna vesselId         0
time             0
cog          51739
sog          51739
rot          51739
heading      51739
navstat      51739
latitude     51739
longitude    51739
split            0
ID               0
dtype: int64
Create features...
df.isna vesselId                                          0
time                                              0
cog                                           51739
sog                                               0
rot                                           51739
navstat                                           0
latitude                                      51739
longitude                                     51739
split                                             0
ID                                                0
time_diff                                       688
long_diff                                     52427
lat_diff                                      52427
navstat_0.0                  

/Users/arthurtestard/vessel_position_forecast/src/data/preprocessing.py:343: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set[features_to_scale] = scaler.transform(test_set[features_to_scale])
100%|██████████| 687/687 [02:30<00:00,  4.55it/s]


Split training and validation sets...
Shapes: (1212155, 8, 12) (1212155, 1, 7) (303039, 8, 12) (303039, 1, 7)
Preprocessing ok... Files stored at: data/preprocessed_v2_seq_8
Number of vessels dropped: 0


In [42]:
X_train[20]

tensor([[-0.0648,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.2224,
         -0.5345,  1.6941,  0.8013, -1.1337],
        [-0.0650,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.1112,
         -0.2161,  1.9823,  0.9829, -0.9667],
        [-0.0650,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.2607,
         -0.4018,  2.1119,  1.2984, -0.4085],
        [-0.0666,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -1.3805,
         -0.4814, -2.2820,  1.1466,  0.7694],
        [-0.0661,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.3840,
          0.5533,  1.5301, -1.0514, -0.9848],
        [ 1.9295,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.4611,
          0.1686,  2.4604, -0.9627, -1.0708],
        [-0.0659,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -1.0972,
          0.1421, -2.5618,  0.7005,  1.2252],
        [-0.0686,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.8028,
          0.7788,  2.2927, -0.

In [3]:
import torch
y = torch.rand(100, 3)
y[:,1].shape

torch.Size([100])

In [43]:
y_train[20]

tensor([[-1.8619e-01,  1.2564e+00,  1.0121e+00, -1.3519e+00,  5.2368e-01,
         -3.0361e-05,  3.0507e-02]])

In [38]:
df_lat_long

,time,vesselId,latitude,longitude
0,2024-01-01 00:00:25,61e9f3a8b937134a3c4bfdf7,-34.74370,-57.85130
1,2024-01-01 00:00:36,61e9f3d4b937134a3c4bff1f,8.89440,-79.47939
2,2024-01-01 00:01:45,61e9f436b937134a3c4c0131,39.19065,-76.47567
3,2024-01-01 00:03:11,61e9f3b4b937134a3c4bfe77,-34.41189,151.02067
4,2024-01-01 00:03:51,61e9f41bb937134a3c4c0087,35.88379,-5.91636
...,...,...,...,...
1522060,2024-05-07 23:59:07,clh6aqawa0002gh0zypfa5dut,52.19131,-5.82223
1522061,2024-05-07 23:59:08,61e9f3aeb937134a3c4bfe43,38.96142,-12.00502
1522062,2024-05-07 23:59:08,61e9f43db937134a3c4c0169,49.71372,-5.22042
1522063,2024-05-07 23:59:08,61e9f469b937134a3c4c029b,38.27895,10.78280


In [37]:
has_nan = torch.isnan(X_train).any()
print(f"Does X_train contain NaN values? {has_nan}")

Does X_train contain NaN values? False


In [15]:
ais_train[ais_train["heading"]==511]


,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId,rot_calculated,rot_category
6,2024-01-01 00:05:40,123.4,0.0,128,511,5,12-16 01:00,40.71466,29.46603,61e9f46bb937134a3c4c02b3,61d38259b7b7526e1adf3a41,53.547782,turning_right
221,2024-01-01 00:35:42,123.4,0.0,128,511,5,12-16 01:00,40.71467,29.46602,61e9f46bb937134a3c4c02b3,61d38259b7b7526e1adf3a41,53.547782,turning_right
306,2024-01-01 00:53:39,123.4,0.0,128,511,5,12-16 01:00,40.71467,29.46604,61e9f46bb937134a3c4c02b3,61d38259b7b7526e1adf3a41,53.547782,turning_right
532,2024-01-01 01:17:43,123.4,0.0,128,511,5,12-16 01:00,40.71467,29.46603,61e9f46bb937134a3c4c02b3,61d38259b7b7526e1adf3a41,53.547782,turning_right
614,2024-01-01 01:35:41,123.4,0.0,128,511,5,12-16 01:00,40.71467,29.46602,61e9f46bb937134a3c4c02b3,61d38259b7b7526e1adf3a41,53.547782,turning_right
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1508421,2024-05-06 21:50:38,45.0,0.0,128,511,15,05-04 13:00,8.98117,-79.57918,61e9f3e8b937134a3c4bff75,61d37cff99db2ccf7339eedd,53.547782,turning_right
1512351,2024-05-07 06:13:37,4.0,0.0,128,511,5,05-06 19:15,51.31070,3.22392,61e9f445b937134a3c4c01a3,61d36f9a0a1807568ff9a156,53.547782,turning_right
1512518,2024-05-07 06:34:37,4.0,0.0,128,511,5,05-06 19:15,51.31072,3.22392,61e9f445b937134a3c4c01a3,61d36f9a0a1807568ff9a156,53.547782,turning_right
1512687,2024-05-07 06:55:37,347.0,0.0,128,511,5,05-06 19:15,51.31068,3.22392,61e9f445b937134a3c4c01a3,61d36f9a0a1807568ff9a156,53.547782,turning_right


In [16]:
ais_test[ais_test["vesselId"]=="61e9f445b937134a3c4c01a3"]

,ID,vesselId,time,scaling_factor
51,51,61e9f445b937134a3c4c01a3,2024-05-08 00:18:07,0.3
191,191,61e9f445b937134a3c4c01a3,2024-05-08 00:39:07,0.3
324,324,61e9f445b937134a3c4c01a3,2024-05-08 00:54:09,0.3
424,424,61e9f445b937134a3c4c01a3,2024-05-08 01:12:09,0.3
570,570,61e9f445b937134a3c4c01a3,2024-05-08 01:33:05,0.3
...,...,...,...,...
46733,46733,61e9f445b937134a3c4c01a3,2024-05-12 12:39:13,0.1
46796,46796,61e9f445b937134a3c4c01a3,2024-05-12 12:49:02,0.1
46932,46932,61e9f445b937134a3c4c01a3,2024-05-12 12:55:49,0.1
47232,47232,61e9f445b937134a3c4c01a3,2024-05-12 13:37:13,0.1


In [71]:
df = ais_test.copy()

In [73]:
schedules = schedules_ft[schedules_ft['arrivalDate'] > '2024-01-01']


In [77]:
len(schedules["vesselId"].unique())

183

In [72]:
# Ensure the 'arrivalDate' column in schedules is in datetime format
schedules_ft['arrivalDate'] = pd.to_datetime(schedules_ft['arrivalDate'].apply(lambda x: str(x).split("+")[0]))
schedules_ = schedules_ft[schedules_ft['arrivalDate'] > '2024-01-01']

# Ensure the 'time' column in ais_train is in datetime format
df['time'] = pd.to_datetime(ais_train['time'])

# Sort both dataframes by 'vesselId' and their respective date columns ('time' and 'arrivalDate')
df_sorted = df.sort_values(by=['time'])
schedules_sorted = schedules_.sort_values(by=['arrivalDate'])

# Perform the merge_asof operation, ensuring both dataframes are sorted and merged in 'forward' direction
merged_df = pd.merge_asof(
    df_sorted,
    schedules_sorted,
    by='vesselId',
    # right_by='vesselId',
    left_on='time',
    right_on='arrivalDate',
    direction='forward'
)

# Display the merged dataframe
print(merged_df)


          ID                  vesselId  ... portLatitude  portLongitude
0          0  61e9f3aeb937134a3c4bfe3d  ...   -17.531111    -149.571389
1          1  61e9f473b937134a3c4c02df  ...          NaN            NaN
2          2  61e9f469b937134a3c4c029b  ...          NaN            NaN
3          3  61e9f45bb937134a3c4c0221  ...          NaN            NaN
4          4  61e9f38eb937134a3c4bfd8d  ...          NaN            NaN
...      ...                       ...  ...          ...            ...
51734  51734  61e9f3a8b937134a3c4bfdf3  ...    51.498889      -2.712222
51735  51735  61e9f3b4b937134a3c4bfe77  ...    31.233000     121.483000
51736  51737  61e9f465b937134a3c4c0269  ...          NaN            NaN
51737  51736  61e9f46cb937134a3c4c02b7  ...          NaN            NaN
51738  51738  61e9f3adb937134a3c4bfe39  ...    50.902500      -1.428889

[51739 rows x 8 columns]


In [68]:
merged_df.sort_values(by=["vesselId", "time"]).isna().sum()

time                   0
cog                    0
sog                    0
rot                    0
heading                0
navstat                0
etaRaw                 0
latitude               0
longitude              0
vesselId               0
portId              1615
arrivalDate      1149615
sailingDate      1149615
portLatitude     1149615
portLongitude    1149615
dtype: int64

In [69]:
ais_train.__len__()

1522065

In [70]:
ais_train.head()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,01-09 23:00,-34.74370,-57.85130,61e9f3a8b937134a3c4bfdf7,61d371c43aeaecc07011a37f
1,2024-01-01 00:00:36,109.6,0.0,-6,347,1,12-29 20:00,8.89440,-79.47939,61e9f3d4b937134a3c4bff1f,634c4de270937fc01c3a7689
2,2024-01-01 00:01:45,111.0,11.0,0,112,0,01-02 09:00,39.19065,-76.47567,61e9f436b937134a3c4c0131,61d3847bb7b7526e1adf3d19
3,2024-01-01 00:03:11,96.4,0.0,0,142,1,12-31 20:00,-34.41189,151.02067,61e9f3b4b937134a3c4bfe77,61d36f770a1807568ff9a126
4,2024-01-01 00:03:51,214.0,19.7,0,215,0,01-25 12:00,35.88379,-5.91636,61e9f41bb937134a3c4c0087,634c4de270937fc01c3a74f3


In [7]:
X_train, X_val, y_train, y_val, test_set, scaler, dropped_vessel_ids = pipeline(seq_len, do_preprocess)

In [16]:
# df_val = ais_train.copy()[ais_train["vesselId"] not in dropped_vessel_ids].sort_values(["vesselId","time"])
df_val = ais_train.copy()[~ais_train["vesselId"].isin(dropped_vessel_ids)].sort_values(["vesselId", "time"])
df_val

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId
131115,2024-01-12 14:07:47,308.1,17.1,-6,316,0,01-08 06:00,7.50361,77.58340,61e9f38eb937134a3c4bfd8b,61d376b393c6feb83e5eb50c
131279,2024-01-12 14:31:00,307.6,17.3,5,313,0,01-14 23:30,7.57302,77.49505,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546
131514,2024-01-12 14:57:23,306.8,16.9,5,312,0,01-14 23:30,7.65043,77.39404,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546
131696,2024-01-12 15:18:48,307.9,16.9,6,313,0,01-14 23:30,7.71275,77.31394,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546
131885,2024-01-12 15:39:47,307.0,16.3,7,313,0,01-14 23:30,7.77191,77.23585,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546
...,...,...,...,...,...,...,...,...,...,...,...
1521244,2024-05-07 22:36:16,324.1,13.5,-2,325,0,05-08 03:00,59.63337,21.43237,clh6aqawa0007gh0z9h6zi9bo,61d373b83aeaecc07011a62b
1521409,2024-05-07 22:57:05,324.2,13.3,-3,326,0,05-08 03:00,59.69588,21.34225,clh6aqawa0007gh0z9h6zi9bo,61d373b83aeaecc07011a62b
1521625,2024-05-07 23:17:54,356.5,12.2,-1,354,0,05-08 03:00,59.76388,21.35317,clh6aqawa0007gh0z9h6zi9bo,61d373b83aeaecc07011a62b
1521821,2024-05-07 23:38:13,52.6,17.3,3,50,0,05-08 03:00,59.83316,21.38489,clh6aqawa0007gh0z9h6zi9bo,61d373b83aeaecc07011a62b


In [40]:
import numpy as np


v_min = np.inf
v_id = ""
for vessel_id in list(ais_test["vesselId"].unique()):
    if len(ais_train[ais_train["vesselId"]==vessel_id]) < v_min:
        v_id = vessel_id
        v_min = len(ais_train[ais_train["vesselId"]==vessel_id])

In [41]:
v_min

328

In [18]:
X_train = torch.rand(100, 48, 10)
y_train = torch.rand(100, 1, 2)

from torch.utils.data import DataLoader, TensorDataset

dataset = TensorDataset(X_train, y_train)
print(dataset.__len__())

AttributeError: 'TensorDataset' object has no attribute 'dataset'

In [20]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

In [22]:
dataloader.dataset.__len__()

100

In [44]:
import numpy as np


v_max = -np.inf
v_id = ""
for vessel_id in list(ais_test["vesselId"].unique()):
    if len(ais_test[ais_test["vesselId"]==vessel_id]) > v_max:
        v_id = vessel_id
        v_max = len(ais_test[ais_test["vesselId"]==vessel_id])

In [45]:
v_max

360

In [47]:
torch.empty(100, 48, 6).mean(dim=1).reshape(100,1,-1).shape

torch.Size([100, 1, 6])

In [34]:
from src.data.features_v0 import create_time_diff_feature

ais_tr_dev = create_time_diff_feature(ais_train)
ais_tr_dev.sort_values(["vesselId", "time"])[["vesselId", "time", "time_diff"]]

,vesselId,time,time_diff
131115,61e9f38eb937134a3c4bfd8b,2024-01-12 14:07:47,1393.0
131279,61e9f38eb937134a3c4bfd8b,2024-01-12 14:31:00,1583.0
131514,61e9f38eb937134a3c4bfd8b,2024-01-12 14:57:23,1285.0
131696,61e9f38eb937134a3c4bfd8b,2024-01-12 15:18:48,1259.0
131885,61e9f38eb937134a3c4bfd8b,2024-01-12 15:39:47,901.0
...,...,...,...
1521244,clh6aqawa0007gh0z9h6zi9bo,2024-05-07 22:36:16,1249.0
1521409,clh6aqawa0007gh0z9h6zi9bo,2024-05-07 22:57:05,1249.0
1521625,clh6aqawa0007gh0z9h6zi9bo,2024-05-07 23:17:54,1219.0
1521821,clh6aqawa0007gh0z9h6zi9bo,2024-05-07 23:38:13,1248.0


In [36]:
39 * 60 + 47 - (18 * 60 + 48)

1259

In [24]:
(X_train.shape[0] + X_val.shape[0]) - (len(ais_train["vesselId"].unique()) - 2) * 48

1455491

In [8]:
dropped_vessel_ids = ['61e9f3adb937134a3c4bfe37', '61e9f3cbb937134a3c4bff09']

In [13]:
len(ais_train[ais_train["vesselId"]=="61e9f3adb937134a3c4bfe37"])

31

In [ ]:
from typing import List
def make_df_val(
        train_set: pd.DataFrame,
        dropped_vessel_ids: List[str],        
        idx: int,

    ) -> pd.DataFrame:
    df_val = train_set.copy()[train_set["vessel_id"] not in dropped_vessel_ids].sort_values("time")
    train_set.iloc[:]

In [ ]:
df_val = train_set.copy()[train_set["vessel_id"] not in dropped_vessel_ids].sort_values("time")


In [6]:
dim_ffn = 126
d_model = 32
activation_dec: Union[str | Callable[[torch.Tensor], torch.Tensor]] = nn.SiLU()

transformer_decoder_params = {
    "d_model": d_model,
    "nhead": 8,
    # "num_encoder_layers": 6,
    # "num_decoder_layers": 2,
    "dim_feedforward": dim_ffn,
    "dropout": 0.1,
    "activation": activation_dec,
    "layer_norm_eps": 0.00001,
    "batch_first": True,
    "norm_first": False,
    # "bias": True,
    "device": DEVICE,
}

In [7]:
class DecoderModel(nn.Module):
    def __init__(
            self,
            decoder_params: Dict[int,Union[int, float, bool]] = transformer_decoder_params, 
            num_features: int = 7, 
            num_outputs: int = 6, 
            num_layers: int = 1,
            act_out: nn.Module | None = None
        ) -> None:
        super().__init__()
        self.emb_layer = nn.Linear(num_features, d_model)
        dec_layer = nn.TransformerDecoderLayer(**decoder_params)
        self.model = nn.TransformerDecoder(dec_layer, num_layers=num_layers)
        self.ffn = nn.Linear(d_model, num_outputs)
        self.act_out = act_out # nn.Sigmoid()
        
    def forward(self, x):
        len_b, len_s, _ = x.shape
        emb = self.emb_layer(x)
        out = self.model(emb, emb)
        out = out[:, -1, :].view(len_b, 1, -1)
        
        if self.act_out:
            return self.act_out(self.ffn(out))
        return self.ffn(out)


model = DecoderModel(act_out=nn.Sigmoid())

In [8]:
from src.train.trainer import Trainer


trainer = Trainer(
    model=model,
    loss=nn.MSELoss(),
    optimizer=torch.optim.AdamW(params=model.parameters()),
    device=DEVICE,
)

In [9]:
X_train = torch.Tensor(X_train).to(DEVICE)
y_train = torch.Tensor(y_train).to(DEVICE)

X_val = torch.Tensor(X_val).to(DEVICE)
y_val = torch.Tensor(y_val).to(DEVICE)

In [10]:

trainer.fit(
    X=X_train,
    y=y_train,
    # X_val=X_val,
    # y_val=y_val,
    epochs=1,
    eval_on_test=True,
    k_folds=0,
)


In [13]:
score = trainer.eval(X_val, y_val)

In [ ]:
import numpy as np

try:
    print("Score on validation set (rmse):", np.sqrt(score))
except:
    print("Score on validation set (rmse):", np.sqrt(score.cpu().numpy()))


In [72]:
def iterative_forecast(seq, model, steps, sequence_length):
    predicted = []
    current_sequence = seq[:sequence_length].reshape(1,sequence_length,7)
    # current_sequence = last_known[-sequence_length:]
    for k in range(steps):
        # next_pred = model.predict(current_sequence.reshape(1, sequence_length, -1))[0]
        x_test = torch.Tensor(current_sequence).to(DEVICE)
        y_pred = model.predict(x_test)[0,0,:]

        predicted.append(y_pred)
        seq[seq_len+k] = np.array([seq[k+1][0], *y_pred])
        
        current_sequence = seq[k+1:k+1+seq_len].reshape(1,seq_len,7)
    print(np.array(predicted).shape)
    return predicted

In [ ]:
# PREDICTION STEP

grouped_test = test_set.groupby("vesselId")

predictions = []

sequence_length = seq_len

for vessel_id, group in tqdm(grouped_test, colour="green"):
    forecast_steps = len(group['time'].values) - seq_len

    last_known_features = group[features_input].values

    future_preds = iterative_forecast(last_known_features, trainer, forecast_steps, sequence_length)
    
    group.loc[group.index[seq_len:],['cog', 'sog', 'rot', 'heading', 'latitude', 'longitude']] = future_preds
    
    group[features_input] = scaler.inverse_transform(group[features_input])
    predictions.append(group.copy())

df_preds = pd.concat(predictions, ignore_index=True)

In [120]:
res = df_preds[["ID","longitude","latitude"]].sort_values("ID")[:51739]
res = res.reset_index().drop(columns="index")

In [ ]:
# SUBMIT RESULT
import uuid
# res = pd.merge(ais_test, df_preds[["vesselId","time", "latitude", "longitude"]],on=["time", "vesselId"], how="left")
res["longitude_predicted"] = res["longitude"]
res["latitude_predicted"] = res["latitude"]
# res = df_preds[["ID"]]
# res["id"] = res["ID"]
res = res.drop(columns=["longitude", "latitude"])

def make_file_name() -> str:
    file_name = str(uuid.uuid4()) + ".csv"
    print(f"Submission file name is: {file_name}")
    return file_name

def submit(forecast: pd.DataFrame, file_name: str = None) -> None:
    sample_submission = pd.read_csv(AIS_SAMPLE_SUBMISSION)
    file_name = file_name if file_name else make_file_name()

    repertory = SUBMISSION_FODLER.joinpath(file_name)
    sample_submission = sample_submission[['ID']].merge(forecast[["ID","longitude_predicted","latitude_predicted"]], on='ID', how='left')
    try:
        sample_submission.to_csv(repertory, index=False)
    except:
        print("Error register file")
        submit(forecast)

submit(res)

In [2]:
import numpy as np

In [7]:
def left_term(x, n , theta, phi):
    return np.exp(-n * x / theta) * x**(n*phi) 

def right_term(alpha, n, theta, phi):
    return (1 - alpha)  * np.exp(-n * phi)* (phi * theta)** (n*phi)

In [8]:
x = 9.32
n = 20
theta = 4.
phi = 2.
alpha = .05

print(left_term(x, n, theta, phi))
print(right_term(alpha, n, theta, phi))

3.4555218024004096e+18
5.364679841538279e+18
